In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import os
import sys
import pathlib
import pickle

sys.path.append('..')

In [3]:
import pandas as pd

In [4]:
from pals.pimp_tools import get_pimp_API_token_from_env, PIMP_HOST, download_from_pimp
from pals.feature_extraction import DataSource
from pals.PALS import PALS
from pals.ORA import ORA
from pals.common import *

2020-03-26 21:26:16.994 | INFO     | pals.reactome:get_neo4j_driver:24 - Created graph database driver for bolt://localhost:7687 (neo4j)


# Beer Analysis

### Load data

Generate token by logging in to PiMP

In [5]:
# username = 'joewandy' # PiMP username
# password = 'enter' # PiMP password
# host = 'localhost:8000' # server address and port
# token = get_authentication_token(host, username, password)

Assume token is stored in environmental variable *PIMP_API_TOKEN*

In [6]:
token = get_pimp_API_token_from_env()

In [7]:
analysis_id = 1321 # example beer analysis

In [8]:
int_df, annotation_df, experimental_design = download_from_pimp(token, PIMP_HOST, analysis_id, 'kegg')

2020-03-26 21:26:17.321 | DEBUG    | pals.pimp_tools:download_from_pimp:119 - Trying to load data from temp file: C:\Users\joewa\AppData\Local\Temp\pimp_analysis_1321.p
2020-03-26 21:26:17.321 | WARNING  | pals.common:load_obj:102 - Old, invalid or missing pickle in C:\Users\joewa\AppData\Local\Temp\pimp_analysis_1321.p. Please regenerate this file.
2020-03-26 21:26:17.323 | DEBUG    | pals.pimp_tools:download_from_pimp:123 - Retrieving data for analysis 1321 from PiMP
2020-03-26 21:26:19.010 | DEBUG    | pals.pimp_tools:get_data:33 - http://polyomics.mvls.gla.ac.uk/export/get_ms1_intensities?analysis_id=1321
2020-03-26 21:26:39.086 | DEBUG    | pals.pimp_tools:get_data:33 - http://polyomics.mvls.gla.ac.uk/export/get_ms1_peaks?analysis_id=1321
2020-03-26 21:26:51.215 | DEBUG    | pals.pimp_tools:get_data:33 - http://polyomics.mvls.gla.ac.uk/export/get_experimental_design?analysis_id=1321
2020-03-26 21:26:51.218 | DEBUG    | pals.pimp_tools:download_from_pimp:132 - Caching analysis data

In [9]:
int_df

,Beer_1_full1.mzXML,Beer_1_full2.mzXML,Beer_1_full3.mzXML,Beer_2_full1.mzXML,Beer_2_full2.mzXML,Beer_2_full3.mzXML,Beer_3_full1.mzXML,Beer_3_full2.mzXML,Beer_3_full3.mzXML,Beer_4_full1.mzXML,Beer_4_full2.mzXML,Beer_4_full3.mzXML
row_id,,,,,,,,,,,,
3033929,2.235291e+09,2.000478e+09,2.170697e+09,2.242760e+09,2.279882e+09,1.959480e+09,2.079356e+09,2.110473e+09,2.243653e+09,1.817065e+09,1.746443e+09,1.779827e+09
3033930,4.433491e+07,4.287387e+07,4.894853e+07,4.760448e+07,4.217280e+07,3.908452e+07,3.825778e+07,3.770192e+07,4.087189e+07,3.330477e+07,3.153630e+07,3.102410e+07
3033931,1.723985e+09,1.764235e+09,1.585143e+09,1.543961e+09,1.579320e+09,1.555666e+09,1.698130e+09,1.481824e+09,1.508645e+09,1.642510e+09,1.723919e+09,1.697806e+09
3033932,6.254237e+08,6.503417e+08,5.914975e+08,4.635929e+08,4.298382e+08,4.038747e+08,4.292837e+08,3.708761e+08,4.778932e+08,3.903165e+08,4.080995e+08,4.309892e+08
3033933,1.075022e+09,9.293474e+08,1.092635e+09,1.130720e+09,1.118146e+09,1.192834e+09,1.231442e+09,1.262046e+09,1.460653e+09,1.009838e+09,9.085111e+08,9.967176e+08
...,...,...,...,...,...,...,...,...,...,...,...,...
3041299,1.431211e+04,6.565678e+03,1.478325e+04,1.620252e+04,1.748920e+04,1.284756e+04,3.306687e+04,2.476216e+04,2.869417e+04,2.231166e+04,2.164017e+04,2.727751e+04
3041300,2.273721e+04,2.905976e+04,2.756565e+04,3.080164e+04,2.427240e+04,2.517871e+04,2.718543e+04,2.905361e+04,3.170420e+04,2.597168e+04,3.066904e+04,2.884984e+04
3041301,1.760107e+04,2.674373e+04,2.165889e+04,2.121242e+04,1.737357e+04,2.039137e+04,2.296079e+04,2.001743e+04,2.664124e+04,1.791359e+04,1.642459e+04,2.349759e+04


In [10]:
annotation_df

,entity_id
row_id,
3033929,C00148
3036581,C00148
3036855,C00148
3038249,C00148
3033929,C00163
...,...
3040926,C20522
3040929,C20582
3041077,C20499


In [11]:
experimental_design

{'comparisons': [{'case': 'beer1', 'control': 'beer2', 'name': 'beer1/beer2'},
  {'case': 'beer3', 'control': 'beer4', 'name': 'beer3/beer4'}],
 'groups': {'beer4': ['Beer_4_full3.mzXML',
   'Beer_4_full2.mzXML',
   'Beer_4_full1.mzXML'],
  'beer3': ['Beer_3_full3.mzXML', 'Beer_3_full2.mzXML', 'Beer_3_full1.mzXML'],
  'beer2': ['Beer_2_full3.mzXML', 'Beer_2_full1.mzXML', 'Beer_2_full2.mzXML'],
  'beer1': ['Beer_1_full2.mzXML', 'Beer_1_full1.mzXML', 'Beer_1_full3.mzXML']}}

### PALS analysis using KEGG database exported from Reactome

In [15]:
ds = DataSource(int_df, annotation_df, experimental_design, DATABASE_REACTOME_KEGG, 
                reactome_species=REACTOME_SPECIES_HOMO_SAPIENS, reactome_metabolic_pathway_only=True)

2020-03-26 21:26:56.217 | DEBUG    | pals.feature_extraction:__init__:42 - Using COMPOUND as database
2020-03-26 21:26:56.218 | DEBUG    | pals.feature_extraction:get_database:130 - Loading ..\pals\data\reactome\metabolic_pathways\COMPOUND\Homo sapiens.json.zip
2020-03-26 21:26:56.236 | DEBUG    | pals.feature_extraction:__init__:55 - Mapping pathway to unique ids
2020-03-26 21:26:56.238 | DEBUG    | pals.feature_extraction:__init__:69 - Creating dataset to pathway mapping
2020-03-26 21:26:57.130 | DEBUG    | pals.feature_extraction:__init__:97 - Computing unique id counts


In [16]:
pals = PALS(ds, plage_weight=5, hg_weight=1)
pathway_df = pals.get_pathway_df()

2020-03-26 21:26:57.197 | DEBUG    | pals.feature_extraction:change_zero_peak_ints:295 - Setting the zero intensity values in the dataframe
2020-03-26 21:26:57.220 | DEBUG    | pals.feature_extraction:standardize_intensity_df:276 - Scaling the data across the sample: zero mean and unit variance
2020-03-26 21:26:57.226 | DEBUG    | pals.PALS:get_plage_activity_df:82 - Mean values of the rows in the DF is [ 0.  0. -0. ... -0. -0. -0.]
2020-03-26 21:26:57.226 | DEBUG    | pals.PALS:get_plage_activity_df:83 - Variance in the rows of the DF is [1. 1. 1. ... 1. 1. 1.]
2020-03-26 21:26:57.313 | DEBUG    | pals.PALS:set_up_resample_plage_p_df:94 - Calculating plage p-values with resampling
2020-03-26 21:26:57.313 | DEBUG    | pals.PALS:set_up_resample_plage_p_df:101 - Comparison beer1/beer2
2020-03-26 21:26:57.314 | DEBUG    | pals.PALS:set_up_resample_plage_p_df:109 - Resampling 0/1000
2020-03-26 21:26:57.427 | DEBUG    | pals.PALS:set_up_resample_plage_p_df:109 - Resampling 100/1000
2020-03-

In [17]:
pathway_df.sort_values('COMPOUND beer1/beer2 comb_p', ascending=True, inplace=True)
pathway_df

,pw_name,beer1/beer2 p-value,beer3/beer4 p-value,unq_pw_F,tot_ds_F,F_coverage,sf,exp_F,Ex_Cov,COMPOUND beer1/beer2 comb_p,COMPOUND beer3/beer4 comb_p
R-HSA-71240,Tryptophan catabolism,0.019149,0.047698,27,14,51.85,0.034569,9.12,33.78,0.008473,0.023205
R-HSA-2024096,HS-GAG degradation,0.014347,0.060139,5,1,20.00,0.874414,1.69,33.80,0.027420,0.097077
R-HSA-71182,Phenylalanine and tyrosine catabolism,0.058698,0.105387,24,13,54.17,0.027395,8.11,33.79,0.027935,0.054381
R-HSA-351143,Agmatine biosynthesis,0.028197,0.091760,5,2,40.00,0.548821,1.69,33.80,0.032383,0.100254
R-HSA-163685,Integration of energy metabolism,0.036262,1.000000,1,1,100.00,0.337802,0.34,34.00,0.032664,1.000000
...,...,...,...,...,...,...,...,...,...,...,...
R-HSA-1855183,"Synthesis of IP2, IP, and Ins in the cytosol",1.000000,0.048672,11,3,27.27,0.779264,3.72,33.82,1.000000,0.070143
R-HSA-381676,Glucagon-like Peptide-1 (GLP1) regulates insul...,1.000000,0.073244,6,1,16.67,0.917414,2.03,33.83,1.000000,0.124730
R-HSA-200425,Import of palmitoyl-CoA into the mitochondrial...,1.000000,0.033261,12,2,16.67,0.952000,4.05,33.75,1.000000,0.070400
R-HSA-8849175,Threonine catabolism,1.000000,0.073244,11,1,9.09,0.990059,3.72,33.82,1.000000,0.166728


### Reactome Expression Analysis

In [66]:
int_df = ds.change_zero_peak_ints()
annot_df = ds.get_annotations()
design = ds.get_experimental_design()

2020-03-26 21:54:35.945 | DEBUG    | pals.feature_extraction:change_zero_peak_ints:295 - Setting the zero intensity values in the dataframe


In [67]:
case = 'beer1'
control = 'beer2'

In [68]:
case_cols = design['groups'][case]
control_cols = design['groups'][control]

In [76]:
df = annot_df.join(int_df).set_index('entity_id').sort_index()

In [81]:
case_df = np.log2(df[case_cols])
control_df = np.log2(df[control_cols])

In [105]:
lfcs = np.mean(case_df, axis=1) - np.mean(control_df, axis=1)

# if there are multiple values for each compound, take the largest fold-change (in either direction) to display
# TODO: we need a better way to do this
fold_changes = {}
for idx, lfc in lfcs.iteritems():
    print(idx, lfc)
    if idx not in fold_changes:
        fold_changes[idx] = lfc
    else:
        if abs(lfc) > abs(fold_changes[idx]):
            fold_changes[idx] = lfc

C00009 -0.021123628019925178
C00009 0.3632305438706638
C00009 -0.05259512035542002
C00009 0.10396288799657327
C00013 0.058142918495278195
C00022 0.3146022204433052
C00022 -0.1419428167406558
C00025 -2.9180947359292233
C00025 -1.3754672587729608
C00025 -1.9038344524184154
C00025 -0.8345895357351694
C00026 0.15449338586888928
C00026 -0.2387897283395155
C00031 0.8740923880434899
C00031 0.27152539694973044
C00031 -1.3902497110592869
C00031 -0.5199093515682129
C00031 0.5394272030803897
C00031 0.44579480263531224
C00031 -0.8783858120981058
C00031 -0.9673291496853267
C00031 0.5822141634218063
C00031 -1.2322503243350482
C00031 0.44579480263531224
C00031 -0.4610797166638321
C00031 -0.3446361356624763
C00031 -0.6532169781561237
C00041 -3.9400049491600377
C00042 0.03086213126071158
C00042 1.763534644422986
C00043 -5.327069508388812
C00047 -2.316052963106614
C00047 6.635615596186399
C00047 -1.6337692660172358
C00047 -2.7693357849902824
C00049 -4.43977739811535
C00049 1.2616504239244435
C00049 -3.7

In [134]:
expression_df = pd.DataFrame.from_dict(fold_changes.items()).set_index(0)
expression_df = expression_df.rename(columns={1: 'log_FC'})
expression_df.index.name = '#id'
expression_df

,log_FC
#id,
C00009,0.363231
C00013,0.058143
C00022,0.314602
C00025,-2.918095
C00026,-0.238790
...,...
C20694,4.540485
PC4093,-1.746863
PC6561,0.256579


In [136]:
# from sklearn import preprocessing

# scaled_data = preprocessing.minmax_scale(expression_df['log_FC'], feature_range=(-1, 1))
# expression_df['log_FC'] = scaled_data  # set scaled data back to the dataframe
# expression_df

,log_FC
#id,
C00009,0.016957
C00013,-0.010123
C00022,0.012641
C00025,-0.274294
C00026,-0.036478
...,...
C20694,0.387731
PC4093,-0.170335
PC6561,0.007491


In [137]:
expression_data = expression_df.to_csv(sep='\t', header=True, index_label='#id', float_format='%.15f')

In [138]:
from urllib.parse import quote

species = REACTOME_SPECIES_HOMO_SAPIENS
encoded_species = quote(species)
encoded_species

'Homo%20sapiens'

In [139]:
import requests

def send_to_reactome(data, encoded_species):
    # refer to https://reactome.org/AnalysisService/#/identifiers/getPostTextUsingPOST
    url = 'https://reactome.org/AnalysisService/identifiers/?interactors=false&species=' + encoded_species + \
          '&sortBy=ENTITIES_PVALUE&order=ASC&resource=TOTAL&pValue=1&includeDisease=true'
    logger.debug('Reactome URL: ' + url)

    # make a POSt request to Reactome Analysis service
    response = requests.post(url, headers={'Content-Type': 'text/plain'}, data=data.encode('utf-8'))
    logger.debug('Response status code = %d' % response.status_code)

    status_code = response.status_code
    if status_code == 200:
        json_response = json.loads(response.text)
    else:
        json_response = None
    return status_code, json_response

def parse_reactome_json(json_response):
    # see https://reactome.org/userguide/analysis for results explanation
    token = json_response['summary']['token']
    pathways = json_response['pathways']

    reactome_url = 'https://reactome.org/PathwayBrowser/#DTAB=AN&ANALYSIS=' + token
    logger.debug('Pathway analysis token: ' + token)
    logger.debug('Pathway analysis URL: ' + reactome_url)

    # https://stackoverflow.com/questions/6027558/flatten-nested-dictionaries-compressing-keys
    pathways_df = pd.io.json.json_normalize(pathways, sep='_')
    return pathways_df, reactome_url, token

In [140]:
status_code, json_response = send_to_reactome(expression_data, encoded_species)

2020-03-26 22:38:53.938 | DEBUG    | __main__:send_to_reactome:7 - Reactome URL: https://reactome.org/AnalysisService/identifiers/?interactors=false&species=Homo%20sapiens&sortBy=ENTITIES_PVALUE&order=ASC&resource=TOTAL&pValue=1&includeDisease=true
2020-03-26 22:38:57.511 | DEBUG    | __main__:send_to_reactome:11 - Response status code = 200


In [141]:
if status_code == 200:
    pathways_df, reactome_url, reactome_token = parse_reactome_json(json_response)

2020-03-26 22:39:01.977 | DEBUG    | __main__:parse_reactome_json:26 - Pathway analysis token: MjAyMDAzMjYyMjM4NTRfNTM0Mg%3D%3D
2020-03-26 22:39:01.980 | DEBUG    | __main__:parse_reactome_json:27 - Pathway analysis URL: https://reactome.org/PathwayBrowser/#DTAB=AN&ANALYSIS=MjAyMDAzMjYyMjM4NTRfNTM0Mg%3D%3D
c:\users\joewa\.virtualenvs\pals-4ls44jcw\lib\site-packages\ipykernel_launcher.py:30: FutureWarning: pandas.io.json.json_normalize is deprecated, use pandas.json_normalize instead


In [142]:
reactome_url

'https://reactome.org/PathwayBrowser/#DTAB=AN&ANALYSIS=MjAyMDAzMjYyMjM4NTRfNTM0Mg%3D%3D'

In [143]:
reactome_token

'MjAyMDAzMjYyMjM4NTRfNTM0Mg%3D%3D'